In [1]:
# SPARK_HOME="/Users/ssv/spark-2.2.0-bin-hadoop2.7"
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
# os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["SPARK_HOME"]='/Users/ssv/spark-2.2.0-bin-hadoop2.7'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.2 (default, Sep 21 2017 18:29:43)
SparkSession available as 'spark'.


In [9]:
ratings_file = 'data/u.data'
films_file = 'data/u.item'

In [20]:
ratings_rdd = sc.textFile(ratings_file)
films_rdd = sc.textFile(films_file)

In [24]:
r_df = ratings_rdd.map(lambda x: x.split('\t')).toDF(schema=['user_id', 'film_id', 'rating', 'hz'])

In [25]:
r_df.show()

+-------+-------+------+---------+
|user_id|film_id|rating|       hz|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



In [27]:
f_df = films_rdd.map((lambda x: (x.split('|')[0],  x.split('|')[1]))).toDF(schema=['film_id', 'film_name'])

In [28]:
f_df.show()

+-------+--------------------+
|film_id|           film_name|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|    GoldenEye (1995)|
|      3|   Four Rooms (1995)|
|      4|   Get Shorty (1995)|
|      5|      Copycat (1995)|
|      6|Shanghai Triad (Y...|
|      7|Twelve Monkeys (1...|
|      8|         Babe (1995)|
|      9|Dead Man Walking ...|
|     10|  Richard III (1995)|
|     11|Seven (Se7en) (1995)|
|     12|Usual Suspects, T...|
|     13|Mighty Aphrodite ...|
|     14|  Postino, Il (1994)|
|     15|Mr. Holland's Opu...|
|     16|French Twist (Gaz...|
|     17|From Dusk Till Da...|
|     18|White Balloon, Th...|
|     19|Antonia's Line (1...|
|     20|Angels and Insect...|
+-------+--------------------+
only showing top 20 rows



In [32]:
r_df.registerTempTable("ratings")
f_df.registerTempTable("films")

In [34]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |    films|       true|
|        |  ratings|       true|
+--------+---------+-----------+



In [61]:
users_count = spark.sql("SELECT count(distinct(user_id)) as u_count FROM ratings").take(1)[0].u_count

In [63]:
films_count = spark.sql("SELECT count(distinct(film_id)) as f_count FROM films").take(1)[0].f_count

In [66]:
print('films count = {}, users_count = {}'.format(films_count, users_count))

films count = 1682, users_count = 943


In [79]:
average_user_ratings = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / users_count, 4)

In [85]:
average_film_ratings = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / films_count, 4)

In [88]:
completness = round(spark.sql("SELECT count(rating) as ratings_count FROM ratings") \
                                    .take(1)[0].ratings_count / (users_count * films_count), 4)

In [89]:
print('average_film_ratings = {}, average_user_ratings = {}, completness = {}'\
                                  .format(average_film_ratings, average_user_ratings, completness))

average_film_ratings = 59.453, average_user_ratings = 106.0445, completness = 0.063


In [108]:
user_user = spark.sql('SELECT user_id as user_id, sum(rating) as ratings_sum, count(rating) as ratings_count,' +
                       ' ROUND (sum(rating) / count(rating), 4) as users_mean_rating FROM ratings GROUP BY user_id')

In [109]:
user_user.show()

+-------+-----------+-------------+-----------------+
|user_id|ratings_sum|ratings_count|users_mean_rating|
+-------+-----------+-------------+-----------------+
|    296|      614.0|          147|           4.1769|
|    467|      162.0|           44|           3.6818|
|    691|      135.0|           32|           4.2188|
|    675|      126.0|           34|           3.7059|
|    829|      227.0|           64|           3.5469|
|    125|      626.0|          182|           3.4396|
|    451|      268.0|           98|           2.7347|
|    800|      105.0|           28|             3.75|
|    853|      122.0|           41|           2.9756|
|    666|      898.0|          245|           3.6653|
|    870|      929.0|          269|           3.4535|
|    919|      753.0|          217|             3.47|
|    926|       66.0|           20|              3.3|
|      7|     1598.0|          403|           3.9653|
|    124|       84.0|           24|              3.5|
|     51|       82.0|       

## user id for ratings prediction:  94

In [110]:
user_user.corr

DataFrame[user_id: string, ratings_sum: double, ratings_count: bigint, users_mean_rating: double]